In [2]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2

%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DAQ = daq.Daq(desc='Testing Daq handling of meta data.')
DAQ.connect_instr('KA3005P', 'ttyUSB0')
DAQ.connect_instr('TDS2024C', 1)

Power supply ID: KORADKA3005PV2.0
Device ttyUSB0 successfully connected and process started.
Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Device C046401 successfully connected and process started.


In [4]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'turn_on')
DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_off')
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_stop', ("SEQuence",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_mode', ("AVErage",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_average_num', (128,))

In [5]:
v = np.linspace(2.0, 30, 57)
chan = {'CH1' : [False, 'Floating Potential in Lab'],
        'CH2' : [False, 'Power Supply Setting']}

#for i in range(len(v)):
for j in [1,2]:
    ch = 'CH{}'.format(j)
    chan[ch][0] = True
    
    DAQ.send_command(DAQ.command_queue['C046401'], 'set_data_source', (ch,))

    DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_record_voltage', (30,))
    
    C_size_os = DAQ.command_queue['C046401'].qsize()
    C_size_ps = DAQ.command_queue['ttyUSB0'].qsize()
    while (C_size_ps != 0) or (C_size_os != 0):
        C_size_os = DAQ.command_queue['C046401'].qsize()
        C_size_ps = DAQ.command_queue['ttyUSB0'].qsize()

    DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_on')
    
    DAQ.send_command(DAQ.command_queue['C046401'], 'wait')
    C_size_os = DAQ.command_queue['C046401'].qsize()
    while C_size_os != 0:
        C_size_os = DAQ.command_queue['C046401'].qsize()
    
    DAQ.send_command(DAQ.command_queue['C046401'], 'save_waveform', (chan,))    
    
    chan[ch][0] = False
        

In [6]:
DAQ.end_dataset()

In [7]:
for key in DAQ.instr:
    for elem in DAQ.instr[key]:
        DAQ.disconnect_instr(key, elem)